<a href="https://colab.research.google.com/github/Xuehui-Jiang/Uni-Mannheim-Master-Thesis/blob/main/Experimental_Evaluation/Experiment_AAUP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cloning the pyRDF2Vec repo
!git clone https://github.com/IBCNServices/pyRDF2Vec.git

Cloning into 'pyRDF2Vec'...
remote: Enumerating objects: 7425, done.
remote: Counting objects: 100% (393/393), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 7425 (delta 242), reused 259 (delta 133), pack-reused 7032
Receiving objects: 100% (7425/7425), 5.33 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (5046/5046), done.


In [ ]:
!pip uninstall community
!pip install python-louvain
!pip install pyRDF2Vec --upgrade

Found existing installation: community 1.0.0b1
Uninstalling community-1.0.0b1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/community-1.0.0b1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/community/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/community/community_louvain.py
    /usr/local/lib/python3.10/dist-packages/community/community_status.py
Proceed (Y/n)? Y
  Successfully uninstalled community-1.0.0b1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install rdflib
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')
import functools
import itertools
from typing import List, Sequence, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rdflib
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC

import sys
sys.path.append('pyRDF2Vec')
sys.path.append('pyRDF2Vec/pyrdf2vec')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.walkers import RandomWalker

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Saving CompleteDataset.tsv to CompleteDataset.tsv
User uploaded file "CompleteDataset.tsv" with length 165644 bytes


In [ ]:
# Load data
complete_df = pd.read_csv('CompleteDataset.tsv', sep='\t')

# Remove rows with duplicate 'DBpedia_URI'
complete_df.drop_duplicates(subset=['Dbpedia_URL'], keep=False, inplace=True)

entities = complete_df['Dbpedia_URL'].tolist()
labels = complete_df['label_salary'].tolist()
reg_targets = complete_df['Average_salary_all_ranks'].tolist()

In [ ]:
print(len(entities))

914


In [ ]:
# Filter out entities and relationships directly related to Cities Dataset
with open("/content/drive/My Drive/dbpedia_files/direct_merged_dbpedia.ttl", "r") as file, open("direct_AAUP_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2:
            # Check if the subject or object of the triple is in our city URL list
            if triple[0].strip('<>') in entities or triple[2].strip('<>') in entities:
                out_file.write(line)

In [ ]:
# Filter out entities and relationships directly or indirectly related to AAUP Dataset
entities_related_to_cities = set(entities)
new_entities = set(entities)
iteration_count = 0

while new_entities and iteration_count < 2:
    temp_entities = set()
    with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file:
        for line in file:
            triple = line.strip().split()
            if len(triple) > 2 and (triple[0].strip('<>') in new_entities or triple[2].strip('<>') in new_entities):
                    temp_entities.add(triple[0].strip('<>'))
                    temp_entities.add(triple[2].strip('<>'))

    new_entities = temp_entities - entities_related_to_cities
    entities_related_to_cities.update(new_entities)
    iteration_count += 1

# Finally, we can write all triples that are directly or indirectly related to entities in the `entities_related_to_cities` collection
with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file, open("indirect_AAUP_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2 and (triple[0].strip('<>') in entities_related_to_cities or triple[2].strip('<>') in entities_related_to_cities):
            out_file.write(line)


In [ ]:
# Define the list of files
files_to_merge = [
    "direct_AAUP_filtered_dbpedia.ttl",
    "indirect_AAUP_filtered_dbpedia.ttl"
]

# Merge files
with open("AAUP_filtered_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open(file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
input_file = "AAUP_filtered_dbpedia.ttl"
output_file = "cleaned_AAUP_filtered_dbpedia.ttl"

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Determine whether it is a comment or a blank line
        if line.startswith("#") or line.strip() == "":
            continue

        # Split triples
        triple = line.strip().split()

        # Make sure this is a complete triple
        if len(triple) > 2:
            # Check whether the object starts with "http://dbpedia.org/resource/"
            if triple[2].startswith("<http://dbpedia.org/resource/"):
                outfile.write(line)

print(f"Finished filtering {input_file} and saved the result to {output_file}")


Finished filtering AAUP_filtered_dbpedia.ttl and saved the result to cleaned_AAUP_filtered_dbpedia.ttl


In [ ]:
!head -n 10 cleaned_AAUP_filtered_dbpedia.ttl

<http://dbpedia.org/resource/Alan_Kay> <http://dbpedia.org/property/workplaces> <http://dbpedia.org/resource/Stanford_University> .
<http://dbpedia.org/resource/Alan_Kay> <http://dbpedia.org/property/workplaces> <http://dbpedia.org/resource/University_of_California,_Los_Angeles> .
<http://dbpedia.org/resource/Alan_Kay> <http://dbpedia.org/property/workplaces> <http://dbpedia.org/resource/Massachusetts_Institute_of_Technology> .
<http://dbpedia.org/resource/André_Weil> <http://dbpedia.org/property/workInstitutions> <http://dbpedia.org/resource/Lehigh_University> .
<http://dbpedia.org/resource/André_Weil> <http://dbpedia.org/property/workInstitutions> <http://dbpedia.org/resource/University_of_Chicago> .
<http://dbpedia.org/resource/Andrew_Wiles> <http://dbpedia.org/property/workInstitutions> <http://dbpedia.org/resource/Princeton_University> .
<http://dbpedia.org/resource/Aage_Bohr> <http://dbpedia.org/property/workInstitutions> <http://dbpedia.org/resource/Columbia_University> .
<http:

In [ ]:
import shutil

destination_path = "/content/drive/My Drive/dbpedia_files"

shutil.copy("cleaned_AAUP_filtered_dbpedia.ttl", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("/content/drive/My Drive/dbpedia_files/cleaned_AAUP_filtered_dbpedia.ttl", fmt='turtle')

kgentities = kg._entities

In [ ]:
for e in entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

http://dbpedia.org/resource/Birmingham%E2%80%93Southern_College
http://dbpedia.org/resource/Goldey%E2%80%93Beacom_College
http://dbpedia.org/resource/University_of_Hawaii%E2%80%93West_Oahu
http://dbpedia.org/resource/Lewis%E2%80%93Clark_State_College
http://dbpedia.org/resource/Rose%E2%80%93Hulman_Institute_of_Technology
http://dbpedia.org/resource/University_of_Michigan%E2%80%93Dearborn
http://dbpedia.org/resource/University_of_Michigan%E2%80%93Flint
http://dbpedia.org/resource/Culver%E2%80%93Stockton_College
http://dbpedia.org/resource/University_of_Missouri%E2%80%93Kansas_City
http://dbpedia.org/resource/University_of_Missouri%E2%80%93St._Louis
http://dbpedia.org/resource/Barber%E2%80%93Scotia_College
http://dbpedia.org/resource/Gardner%E2%80%93Webb_University
http://dbpedia.org/resource/University_of_Nebraska%E2%80%93Lincoln
http://dbpedia.org/resource/Colby%E2%80%93Sawyer_College
http://dbpedia.org/resource/Richard_Stockton_College_of_New_Jersey
http://dbpedia.org/resource/Baldwin

In [ ]:
# Entities not in KG
absent_entities = [
    "http://dbpedia.org/resource/Birmingham%E2%80%93Southern_College",
    "http://dbpedia.org/resource/Goldey%E2%80%93Beacom_College",
    "http://dbpedia.org/resource/University_of_Hawaii%E2%80%93West_Oahu",
    "http://dbpedia.org/resource/Lewis%E2%80%93Clark_State_College",
    "http://dbpedia.org/resource/Rose%E2%80%93Hulman_Institute_of_Technology",
    "http://dbpedia.org/resource/University_of_Michigan%E2%80%93Dearborn",
    "http://dbpedia.org/resource/University_of_Michigan%E2%80%93Flint",
    "http://dbpedia.org/resource/Culver%E2%80%93Stockton_College",
    "http://dbpedia.org/resource/University_of_Missouri%E2%80%93Kansas_City",
    "http://dbpedia.org/resource/University_of_Missouri%E2%80%93St._Louis",
    "http://dbpedia.org/resource/Barber%E2%80%93Scotia_College",
    "http://dbpedia.org/resource/Gardner%E2%80%93Webb_University",
    "http://dbpedia.org/resource/University_of_Nebraska%E2%80%93Lincoln",
    "http://dbpedia.org/resource/Colby%E2%80%93Sawyer_College",
    "http://dbpedia.org/resource/Richard_Stockton_College_of_New_Jersey",
    "http://dbpedia.org/resource/Baldwin%E2%80%93Wallace_College",
    "http://dbpedia.org/resource/Gwynedd%E2%80%93Mercy_College",
    "http://dbpedia.org/resource/Carson%E2%80%93Newman_College",
    "http://dbpedia.org/resource/Freed%E2%80%93Hardeman_University",
    "http://dbpedia.org/resource/Hardin%E2%80%93Simmons_University",
    "http://dbpedia.org/resource/University_of_Houston%E2%80%93Downtown",
    "http://dbpedia.org/resource/University_of_Houston%E2%80%93Victoria",
    "http://dbpedia.org/resource/Hampden%E2%80%93Sydney_College",
    "http://dbpedia.org/resource/Randolph%E2%80%93Macon_College",
    "http://dbpedia.org/resource/Mount_Mary_College",
    "http://dbpedia.org/resource/University_of_Wisconsin-Madison_School_of_Music",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93Green_Bay",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93Stout",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93Eau_Claire",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93La_Crosse",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93Platteville",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93River_Falls",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93Superior",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93Whitewater",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93Parkside",
    "http://dbpedia.org/resource/University_of_Wisconsin%E2%80%93Oshkosh",
    "http://dbpedia.org/resource/Alderson%E2%80%93Broaddus_College"
]

# Filter out absent entities
filtered_entities = [e for e in entities if e not in absent_entities]
filtered_labels = [label for entity, label in zip(entities, labels) if entity not in absent_entities]
filtered_reg_targets = [reg_targets for entity, reg_targets in zip(entities, reg_targets) if entity not in absent_entities]

In [ ]:
print(len(filtered_entities))
print(len(filtered_labels))

877
877


In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

Prepare for Knowledge Graph

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
import networkx as nx

In [ ]:
# Load the graph from the input file
graph = Graph()
graph.parse("/content/drive/My Drive/dbpedia_files/cleaned_AAUP_filtered_dbpedia.ttl", format="turtle")

<Graph identifier=N559689b255744d83841ccfb1d715a442 (<class 'rdflib.graph.Graph'>)>

In [ ]:
G = nx.DiGraph()  # Create a directed graph

for s, p, o in graph:
    G.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G.nodes())
num_edges = len(G.edges())
print("The DBpedia knowledge graph as of 2016-10 has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The DBpedia knowledge graph as of 2016-10 has 3239817 nodes and 8439719 edges.


Classification & Regression tasks based graph embeddings without SA

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim)
        #embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('pure_class_reg_results.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.476620
1           cbow         50                 C4.5  Accuracy   0.419527
2           cbow         50            KNN (k=3)  Accuracy   0.435554
3           cbow         50                  SVM  Accuracy   0.558699
4           cbow         50    Linear Regression      RMSE  84.822852


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with BFS

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving main.py to main.py
Saving sa_helper.py to sa_helper.py


In [ ]:
from main import *
from sa_helper import *

In [ ]:
# Count the neighbor node number for each nodes in the graph
neighbor_counts = {node: len(list(G.neighbors(node))) for node in G.nodes()}

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub.nodes())
num_edges = len(G_sub.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 74394 nodes and 106590 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities[240:360])

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels[240:360], cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets[240:360], cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric       Score
0           cbow         50          Naive Bayes  Accuracy    0.416667
1           cbow         50                 C4.5  Accuracy    0.500000
2           cbow         50            KNN (k=3)  Accuracy    0.466667
3           cbow         50                  SVM  Accuracy    0.491667
4           cbow         50    Linear Regression      RMSE  121.287758


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 74394 nodes and 106590 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from google.colab import files
files.download('BFS_subgraph_tf.nt')

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.507471
1           cbow         50                 C4.5  Accuracy   0.434444
2           cbow         50            KNN (k=3)  Accuracy   0.457197
3           cbow         50                  SVM  Accuracy   0.551881
4           cbow         50    Linear Regression      RMSE  91.356860


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 74394 nodes and 106590 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.511951
1           cbow         50                 C4.5  Accuracy   0.422923
2           cbow         50            KNN (k=3)  Accuracy   0.449216
3           cbow         50                  SVM  Accuracy   0.551881
4           cbow         50    Linear Regression      RMSE  87.142372


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 74394 nodes and 106590 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.514263
1           cbow         50                 C4.5  Accuracy   0.437631
2           cbow         50            KNN (k=3)  Accuracy   0.460567
3           cbow         50                  SVM  Accuracy   0.547335
4           cbow         50    Linear Regression      RMSE  88.080583


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with DFS

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub_ff = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub_ff.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ff.nodes())
num_edges = len(G_sub_ff.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 74393 nodes and 106590 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.522218
1           cbow         50                 C4.5  Accuracy   0.420637
2           cbow         50            KNN (k=3)  Accuracy   0.424138
3           cbow         50                  SVM  Accuracy   0.555290
4           cbow         50    Linear Regression      RMSE  92.954723


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.1, beta=0.009, max_depth=2, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 74393 nodes and 106590 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.506322
1           cbow         50                 C4.5  Accuracy   0.435410
2           cbow         50            KNN (k=3)  Accuracy   0.463989
3           cbow         50                  SVM  Accuracy   0.546186
4           cbow         50    Linear Regression      RMSE  89.906614


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 74393 nodes and 106590 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.510828
1           cbow         50                 C4.5  Accuracy   0.415152
2           cbow         50            KNN (k=3)  Accuracy   0.449086
3           cbow         50                  SVM  Accuracy   0.546199
4           cbow         50    Linear Regression      RMSE  88.259783


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 74393 nodes and 106590 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.517672
1           cbow         50                 C4.5  Accuracy   0.411533
2           cbow         50            KNN (k=3)  Accuracy   0.445742
3           cbow         50                  SVM  Accuracy   0.558686
4           cbow         50    Linear Regression      RMSE  87.583045
